In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

In [2]:
def build_model(hp):
    
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    
    model = Sequential([
        Dense(units, activation="relu"),
        Dense(10, activation="softmax")
    ])
    
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [5]:
(x_train, y_train), (x_test, y_test) = load_data()

In [6]:
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255.0
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255.0

x_train_full = x_train[:]
y_train_full = y_train[:]

In [7]:
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

In [11]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [12]:
def get_best_epoch(hp):
    
    model = build_model(hp)
    callbacks = [EarlyStopping(monitor="val_loss", mode="min", patience=10)]
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=128, callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    
    return best_epoch

In [13]:
def get_best_trained_model(hp):

    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(x_train_full, y_train_full, batch_size=128, epochs=int(best_epoch * 1.2))
    
    return model

In [14]:
best_models = []

for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 1s 3ms/step - loss: 0.4242 - accuracy: 0.8855 - val_loss: 0.2246 - val_accuracy: 0.9399
Epoch 2/100
391/391 [==============================] - 1s 3ms/step - loss: 0.2137 - accuracy: 0.9385 - val_loss: 0.1711 - val_accuracy: 0.9522
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1626 - accuracy: 0.9535 - val_loss: 0.1590 - val_accuracy: 0.9556
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1329 - accuracy: 0.9620 - val_loss: 0.1251 - val_accuracy: 0.9635
Epoch 5/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1127 - accuracy: 0.9677 - val_loss: 0.1190 - val_accuracy: 0.9659
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0976 - accuracy: 0.9717 - val_loss: 0.1128 - val_accuracy: 0.9675
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0859 - accuracy: 0.9751 - val_loss: 0.1092 - val_accuracy: 0.9673

Epoch 29/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0254 - accuracy: 0.9936 - val_loss: 0.1148 - val_accuracy: 0.9726
Epoch 30/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0243 - accuracy: 0.9938 - val_loss: 0.1123 - val_accuracy: 0.9722
Best epoch: 20
Epoch 1/24
469/469 [==============================] - 1s 2ms/step - loss: 0.4170 - accuracy: 0.8870
Epoch 2/24
469/469 [==============================] - 1s 2ms/step - loss: 0.2173 - accuracy: 0.9386
Epoch 3/24
469/469 [==============================] - 1s 3ms/step - loss: 0.1704 - accuracy: 0.9506
Epoch 4/24
469/469 [==============================] - 1s 3ms/step - loss: 0.1432 - accuracy: 0.9589
Epoch 5/24
469/469 [==============================] - 1s 3ms/step - loss: 0.1232 - accuracy: 0.9638
Epoch 6/24
469/469 [==============================] - 1s 2ms/step - loss: 0.1099 - accuracy: 0.9681
Epoch 7/24
469/469 [==============================] - 1s 3ms/step - loss: 0.0985 - accuracy: 0.97

469/469 [==============================] - 1s 2ms/step - loss: 0.0585 - accuracy: 0.9825
Epoch 14/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0540 - accuracy: 0.9844
Epoch 15/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0503 - accuracy: 0.9858
Epoch 16/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.9866
Epoch 17/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0437 - accuracy: 0.9876
Epoch 18/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0411 - accuracy: 0.9879
Epoch 19/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0378 - accuracy: 0.9895
Epoch 20/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0352 - accuracy: 0.9902
Epoch 21/22
469/469 [==============================] - 1s 2ms/step - loss: 0.0333 - accuracy: 0.9908
Epoch 22/22
313/313 [==============================] - 1s 2ms/step - loss: 0.0946 - accuracy: 0.9719
Ep

In [17]:
best_models = tuner.get_best_models(top_n)

### Speeding up training on GPU with mixed precision

#### Understanding floating-point precision

In [0]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

In [0]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

#### Mixed-precision training in practice

In [0]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")